<a href="https://colab.research.google.com/github/qihua22/awesome-public-datasets/blob/master/%E7%BF%BB%E8%AF%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/moses-smt/mosesdecoder.git

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148459, done.
remote: Counting objects: 100% (887/887), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 148459 (delta 565), reused 797 (delta 517), pack-reused 147572
Receiving objects: 100% (148459/148459), 129.99 MiB | 18.31 MiB/s, done.
Resolving deltas: 100% (114591/114591), done.


In [2]:
!wget https://data.statmt.org/news-commentary/v18.1/training/news-commentary-v18.en-zh.tsv.gz -P /content/nmt/data/v18news
!gunzip /content/nmt/data/v18news/news-commentary-v18.en-zh.tsv.gz


--2024-06-27 02:53:24--  https://data.statmt.org/news-commentary/v18.1/training/news-commentary-v18.en-zh.tsv.gz
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52457479 (50M) [application/x-gzip]
Saving to: ‘/content/nmt/data/v18news/news-commentary-v18.en-zh.tsv.gz’

news-commentary-v18 100%[===================>]  50.03M  20.3MB/s    in 2.5s    

2024-06-27 02:53:27 (20.3 MB/s) - ‘/content/nmt/data/v18news/news-commentary-v18.en-zh.tsv.gz’ saved [52457479/52457479]



In [ ]:
import sys
import os

# 将路径添加到sys.path
sys.path.append('/content/nmt')

In [4]:
!python3 /content/nmt/utils/cut2.py /content/nmt/data/v18news/news-commentary-v18.en-zh.tsv /content/nmt/data/v18news/


In [5]:
!perl /content/mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l en < /content/nmt/data/v18news/raw.en > /content/nmt/data/v18news/norm.en
!perl /content/mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l zh < /content/nmt/data/v18news/raw.zh > /content/nmt/data/v18news/norm.zh


In [6]:
!python3 -m jieba -d " " /content/nmt/data/v18news/norm.zh > /content/nmt/data/v18news/norm.seg.zh


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.713 seconds.
Prefix dict has been built successfully.


In [7]:
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl -l en < /content/nmt/data/v18news/norm.en > /content/nmt/data/v18news/norm.tok.en
!/content/mosesdecoder/scripts/tokenizer/tokenizer.perl -l zh < /content/nmt/data/v18news/norm.seg.zh > /content/nmt/data/v18news/norm.seg.tok.zh


Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: zh
Number of threads: 1


In [16]:
# Step 1: Truecasing
!/content/mosesdecoder/scripts/recaser/train-truecaser.perl --model /content/nmt/models/v18news/truecase-model.en --corpus /content/nmt/data/v18news/norm.tok.en

!/content/mosesdecoder/scripts/recaser/truecase.perl --model /content/nmt/models/v18news/truecase-model.en < /content/nmt/data/v18news/norm.tok.en > /content/nmt/data/v18news/norm.tok.true.en



In [9]:
!head /content/nmt/data/v18news/norm.tok.true.en

In [10]:
!git clone https://github.com/rsennrich/subword-nmt.git

Cloning into 'subword-nmt'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 600 (delta 10), reused 12 (delta 4), pack-reused 576
Receiving objects: 100% (600/600), 253.58 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (359/359), done.


In [17]:
!python /content/subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input /content/nmt/data/v18news/norm.tok.true.en  -s 32000 -o /content/nmt/models/v18news/bpecode.en --write-vocabulary /content/nmt/models/v18news/voc.en
!python /content/subword-nmt/subword_nmt/apply_bpe.py -c /content/nmt/models/v18news/bpecode.en --vocabulary /content/nmt/models/v18news/voc.en < /content/nmt/data/v18news/norm.tok.true.en > /content/nmt/data/v18news/norm.tok.true.bpe.en

!python /content/subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input /content/nmt/data/v18news/norm.seg.tok.zh  -s 32000 -o /content/nmt/models/v18news/bpecode.zh --write-vocabulary /content/nmt/models/v18news/voc.zh
!python /content/subword-nmt/subword_nmt/apply_bpe.py -c /content/nmt/models/v18news/bpecode.zh --vocabulary /content/nmt/models/v18news/voc.zh < /content/nmt/data/v18news/norm.seg.tok.zh > /content/nmt/data/v18news/norm.seg.tok.bpe.zh


100% 32000/32000 [01:31<00:00, 351.36it/s]
100% 32000/32000 [01:32<00:00, 346.42it/s] 


In [18]:
!mv /content/nmt/data/v18news/norm.seg.tok.bpe.zh /content/nmt/data/v18news/toclean.zh
!mv /content/nmt/data/v18news/norm.tok.true.bpe.en /content/nmt/data/v18news/toclean.en
!/content/mosesdecoder/scripts/training/clean-corpus-n.perl /content/nmt/data/v18news/toclean zh en /content/nmt/data/v18news/clean 1 256


clean-corpus.perl: processing /content/nmt/data/v18news/toclean.zh & .en to /content/nmt/data/v18news/clean, cutoff 1-256, ratio 9
..........(100000)..........(200000)..........(300000)..........(400000).....
Input sentences: 454824  Output sentences:  442874


In [2]:
# 重新克隆 Fairseq 仓库
!git clone https://github.com/pytorch/fairseq
%cd fairseq

# 安装依赖
!pip install omegaconf
!pip install -r requirements.txt

# 安装 Fairseq
!python setup.py install

# 验证安装
!fairseq-preprocess --help



Cloning into 'fairseq'...
remote: Enumerating objects: 35203, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 35203 (delta 66), reused 80 (delta 48), pack-reused 35083
Receiving objects: 100% (35203/35203), 25.23 MiB | 22.04 MiB/s, done.
Resolving deltas: 100% (25556/25556), done.
/content/fairseq
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html 

In [3]:
!fairseq-preprocess \
    --source-lang zh --target-lang en \
    --trainpref /content/nmt/data/v18news/clean/train \
    --validpref /content/nmt/data/v18news/clean/valid \
    --testpref /content/nmt/data/v18news/clean/test \
    --destdir /content/nmt/data-bin/v18news \
    --workers 4


2024-06-27 04:52:19.641225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 04:52:19.641300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 04:52:19.642990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-27 04:52:19.652077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 04:52:21.124738: W tensorflow/compiler/tf2

In [5]:
!pip install hydra-core
!fairseq-preprocess \
    --source-lang zh --target-lang en \
    --trainpref /content/nmt/data/v18news/clean/train \
    --validpref /content/nmt/data/v18news/clean/valid \
    --testpref /content/nmt/data/v18news/clean/test \
    --destdir /content/nmt/data-bin/v18news \
    --workers 4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 1.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 0.12.2 requires hydra-core<1.1,>=1.0.7, but you have hydra-core 1.3.2 which is incompatible.
fairseq 0.12.2 requires omegaconf<2.1, but you have omegaconf 2.3.0 which is incompatible.
2024-06-27 04:53:25.454375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 04:53:25.454438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 04:53:25.456111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: At

In [6]:
!pip install portalocker


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sacrebleu 2.4.2 requires colorama, which is not installed.


In [7]:
!pip install colorama
!pip install portalocker


In [8]:
!fairseq-preprocess --source-lang zh --target-lang en \
    --trainpref /content/nmt/data/v18news/train --validpref /content/nmt/data/v18news/valid --testpref /content/nmt/data/v18news/test \
    --destdir /content/nmt/data/v18news/data-bin


2024-06-27 04:56:37.671799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 04:56:37.671871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 04:56:37.674211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-27 04:56:37.686380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 04:56:39.467293: W tensorflow/compiler/tf2

In [3]:
!pip install 'omegaconf<2.1'


  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.1.2
    Uninstalling omegaconf-2.1.2:
      Successfully uninstalled omegaconf-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 0.12.2 requires hydra-core<1.1,>=1.0.7, but you have hydra-core 1.0.3 which is incompatible.


In [4]:
!pip install 'hydra-core>=1.0.7,<1.1'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: hydra-core
    Found existing installation: hydra-core 1.0.3
    Uninstalling hydra-core-1.0.3:
      Successfully uninstalled hydra-core-1.0.3


In [ ]:
!CUDA_VISIBLE_DEVICES=0,1,2,3  fairseq-train /content/nmt/data/v18news/data-bin --arch transformer \
	--source-lang zh --target-lang en  \
    --optimizer adam  --lr 0.001 --adam-betas '(0.9, 0.98)' \
    --lr-scheduler inverse_sqrt --max-tokens 4096  --dropout 0.3 \
    --criterion label_smoothed_cross_entropy  --label-smoothing 0.1 \
    --max-update 200000  --warmup-updates 4000 --warmup-init-lr '1e-07' \
    --keep-last-epochs 10 --num-workers 8 \
	--save-dir /content/nmt/models/v18news/checkpoints &


2024-06-27 06:32:02.184487: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 06:32:02.184633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 06:32:02.341429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-27 06:32:02.639780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 06:32:16.837786: W tensorflow/compiler/tf2

2024-06-27 05:06:30 | INFO | fairseq.tasks.fairseq_task | reuse_dataloader = True
2024-06-27 05:06:30 | INFO | fairseq.tasks.fairseq_task | rebuild_batches = False
2024-06-27 05:06:30 | INFO | fairseq.tasks.fairseq_task | creating new batches for epoch 1
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
2024-06-27 05:06:33 | INFO | fairseq.d

In [ ]:
# 挂载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

# 创建目标目录
import os
destination_dir = '/content/drive/MyDrive/nmt_backup'
os.makedirs(destination_dir, exist_ok=True)

# 复制文件
import shutil
source_dir = '/content/nmt'
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

# 确认文件复制成功
os.listdir(destination_dir)


Mounted at /content/drive


['.ipynb_checkpoints', 'models', 'utils', 'data']